##  감성체인 생성

In [7]:
import os
import time
import random
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain  # LLMChain을 사용하여 Chain을 구성합니다
import logging
import traceback

In [8]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [9]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0.3)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate.from_template(
    """你是一名研究中国的社会科学家，你的任务是分析中国新闻对“美日韩”三边关系的报道，并从中国的视角分析这些报道题目的情感**。

    **评分标准**：
    - 强烈负面 (-5): 极端消极
    - 负面 (-3 ~ -4): 较为消极
    - 轻微负面 (-1 ~ -2): 略微消极
    - 中立 (0): 纯粹的事实陈述，不带有任何情感倾向。
    - 轻微正面 (+1 ~ +2): 略微积极
    - 正面 (+3 ~ +4): 较为积极
    - 强烈正面 (+5): 极端积极

    **请严格分析以下新闻文章的题目，并仅返回每个题目的情感分数，每行一个分数，不要包含任何额外的文本、注释或解释。所有提供的题目都必须进行情感分析，不得遗漏任何文章。**

    **新闻题目**
    {title}
"""
)

In [10]:
# 체인 구성
sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)

## 데이터 로드

In [14]:
import json

# JSON 파일 로드
file_path = "../../data/title/2020.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 {len(data)}개의 기사 로드 완료")

총 11개의 기사 로드 완료


## 감성분석 수행 함수

In [12]:
def analyze_sentiment(article):
    """기사 제목만 입력하고 감성 점수를 분석하는 함수"""
    try:
        delay_time = random.uniform(10, 15)  # 10~15초 랜덤 딜레이
        time.sleep(delay_time)

        # 제목만 전달
        input_data = {"title": article["title"]}

        print(f"Sending request for '{article['title']}': {input_data}")
        response = sentiment_chain.invoke(input_data)
        print(f"Response for '{article['title']}': {response}")

        if isinstance(response, dict) and 'text' in response:
            article["sentiment_responses"] = response['text'].strip()  # 감성 분석 결과 저장
        else:
            print(f"Error: 'text' key not found in response for '{article['title']}'")
            article["sentiment_responses"] = ""

        return article
    except Exception as e:
        print(f"Exception occurred while processing '{article['title']}': {e}")
        article["sentiment_responses"] = ""
        return article

def retry_request(article, max_retries=2, delay_factor=2):
    """백오프 전략을 적용하여 감성 분석 요청을 재시도하는 함수"""
    result = article.copy()  # 기본값 설정
    for attempt in range(max_retries):
        result = analyze_sentiment(article)
        if result["sentiment_responses"] != "":
            return result
        else:
            print(f"Retry {attempt + 1}/{max_retries} for '{article['title']}' due to error.")
            time.sleep(random.uniform(30, 40) * delay_factor)
            delay_factor *= 2
    return result

def run_sentiment_analysis(data, output_file):
    """JSON 파일을 로드하여 감성 분석을 실행하고 결과를 저장하는 함수"""
    results = []

    for article in data:
        processed_article = retry_request(article)
        results.append(processed_article)

        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=4)
        print(f"'{article['title']}' 분석 완료.")

    print(f"전체 기사 감성 분석 완료. 결과 저장: {output_file}")


In [15]:
output_path = "../../data/title/results/2020.json"
run_sentiment_analysis(data, output_path)

Sending request for '美日韩外长会举行,半岛问题、中东局势方面三国并不齐心': {'title': '美日韩外长会举行,半岛问题、中东局势方面三国并不齐心'}


2025-03-19 15:50:17,541 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response for '美日韩外长会举行,半岛问题、中东局势方面三国并不齐心': {'title': '美日韩外长会举行,半岛问题、中东局势方面三国并不齐心', 'text': '-1'}
'美日韩外长会举行,半岛问题、中东局势方面三国并不齐心' 분석 완료.
Sending request for '日韩外长在美进行会谈双方矛盾仍然难以弥合': {'title': '日韩外长在美进行会谈双方矛盾仍然难以弥合'}


2025-03-19 15:50:32,543 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response for '日韩外长在美进行会谈双方矛盾仍然难以弥合': {'title': '日韩外长在美进行会谈双方矛盾仍然难以弥合', 'text': '-2'}
'日韩外长在美进行会谈双方矛盾仍然难以弥合' 분석 완료.
Sending request for '美日韩外长会举行，半岛问题、中东局势方面三国并不齐心': {'title': '美日韩外长会举行，半岛问题、中东局势方面三国并不齐心'}


...

Response for '日韩军事合作步履蹒跚 - 人民日报 - 人民网': {'title': '日韩军事合作步履蹒跚 - 人民日报 - 人民网', 'text': '-2'}
'日韩军事合作步履蹒跚 - 人民日报 - 人民网' 분석 완료.
Sending request for '美欲建“印太版北约”仍以自我为中心': {'title': '美欲建“印太版北约”仍以自我为中心'}


2025-03-19 15:52:13,304 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response for '美欲建“印太版北约”仍以自我为中心': {'title': '美欲建“印太版北约”仍以自我为中心', 'text': '-3'}
'美欲建“印太版北约”仍以自我为中心' 분석 완료.
Sending request for '急于巩固同盟关系？日韩领导人同日与拜登通话': {'title': '急于巩固同盟关系？日韩领导人同日与拜登通话'}


2025-03-19 15:52:28,625 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response for '急于巩固同盟关系？日韩领导人同日与拜登通话': {'title': '急于巩固同盟关系？日韩领导人同日与拜登通话', 'text': '-1'}
'急于巩固同盟关系？日韩领导人同日与拜登通话' 분석 완료.
전체 기사 감성 분석 완료. 결과 저장: ../../data/title/results/2020.json
